In [13]:
import googlemaps
import folium
import pandas as pd
import numpy as np
import heapq
from haversine import haversine #거리구하기
from sklearn.preprocessing import minmax_scale #정규화

##data 불러오기
total_df = pd.read_csv("all_df.csv", encoding = "EUC-KR" )
df = total_df.rename(index=str, columns={"Unnamed: 0" : "ID"})
df = df.drop(columns = ['경유여부','거리'])
print("제거전", len(df))
l_max = 125
df  = df[ df['총밝기'] < l_max]
print("제거후", len(df))

##출발지, 도착지
sp = (37.506059, 127.036863) #출발지 37.506059, 127.036863
ep = (37.509122, 127.043816) #도착지 37.509122, 127.043816

df = df.append(df.from_dict({'ID' : -1,'point_ID':'Point_SP', '위도':sp[0], '경도':sp[1], '밝기':-l_max
                             , '분류':'출발지', '명칭':'출발지', '총밝기' : -l_max }, orient = 'index').T)

df = df.append(df.from_dict({'ID' : len(df), 'point_ID':'Point_EP', '위도':ep[0], '경도':ep[1], '밝기':l_max
                             , '분류':'도착지', '명칭':'도착지', '총밝기' : l_max}, orient = 'index').T)

df = df.sort_values(by='ID').reset_index(drop=True)
df['ID'] = df.index.values
# #df['ID'] = df.index.valuesd
# df = df.set_index('ID')
df.to_csv("all_df2.csv", mode='w', encoding='ms949') 
df.head(5)

제거전 2090
제거후 2004


,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
0,0,Point_SP,127.037,출발지,-125,출발지,37.5061,-125
1,1,Point_5,127.031,역삼1동832-25,5,보안등,37.4928,10
2,2,Point_6,127.032,역삼1동832-27,5,보안등,37.4926,50
3,3,Point_7,127.032,역삼1동832-28,5,보안등,37.4925,50
4,4,Point_8,127.032,역삼1동832-28,5,보안등,37.4925,50


In [14]:
#총 거리
d_m = haversine(sp, ep, unit='m') # in meters
d_km = haversine(sp, ep,  unit='m')  # in kilometers

#block단위 
# 경유지가 총 3개까지만 가능하니까 3개의 반경으로 나누자. 700 이상인 경우는 api다시 받기 (추후에)
dvd = d_m / 4
print( "단위거리(dvd) : ",  dvd, ", 총거리(d_m) : " , d_m)

단위거리(dvd) :  175.38377731330854 , 총거리(d_m) :  701.5351092532342


In [15]:
#!pip install geopy
import csv, geopy.distance, collections, heapq

Location = collections.namedtuple("Location", "ID point_ID 경도 명칭 밝기 분류 위도 총밝기".split())
data = {}
with open("all_df2.csv") as f:
    r = csv.DictReader(f)
    for d in r:
        i, pi, x, y, c, n,l,tl  = int(d["ID"]), d["point_ID"], d["분류"], d["명칭"], d["위도"], d["경도"], d["밝기"], d["총밝기"]
        data[i] = Location(i, pi, x, y, c, n,l,tl)
data

{0: Location(ID=0, point_ID='Point_SP', 경도='출발지', 명칭='출발지', 밝기='37.506059', 분류='127.036863', 위도='-125', 총밝기='-125'),
 1: Location(ID=1, point_ID='Point_5', 경도='보안등', 명칭='역삼1동832-25', 밝기='37.49283351', 분류='127.03141329399999', 위도='5', 총밝기='10'),
 2: Location(ID=2, point_ID='Point_6', 경도='보안등', 명칭='역삼1동832-27', 밝기='37.4926316591', 분류='127.031508196', 위도='5', 총밝기='50'),
 3: Location(ID=3, point_ID='Point_7', 경도='보안등', 명칭='역삼1동832-28', 밝기='37.4924802706', 분류='127.03158050200001', 위도='5', 총밝기='50'),
 4: Location(ID=4, point_ID='Point_8', 경도='보안등', 명칭='역삼1동832-28', 밝기='37.4924802706', 분류='127.03158050200001', 위도='5', 총밝기='50'),
 5: Location(ID=5, point_ID='Point_9', 경도='보안등', 명칭='역삼1동832-29', 밝기='37.4923433006', 분류='127.031643768', 위도='5', 총밝기='75'),
 6: Location(ID=6, point_ID='Point_12', 경도='보안등', 명칭='역삼1동837-33', 밝기='37.4915755437', 분류='127.03200981399999', 위도='5', 총밝기='65'),
 7: Location(ID=7, point_ID='Point_13', 경도='보안등', 명칭='역삼1동837-15', 밝기='37.4918998658', 분류='127.03215921399999', 위도

In [205]:
#출발지-도착지 총거리계산 
def calcH(start, end):
    coords_1 = (data[start].latitude, data[start].longitude)
    coords_2 = (data[end].latitude, data[end].longitude)
    distance = (geopy.distance.vincenty(coords_1, coords_2)).m
    return distance

##주변 points 얻어오기 ( 정렬기준은 각점들과 EP까지의 거리인 H)
def getneighbors(point, n=6):
    #return sorted(data.values(), key=lambda x: calcH(startpoint, x.ID))[1:n+1]
     #[1:n+1] : 자신을 제외한 주위의 점들 중에서 EP까지의 거리인 H가 가장 작은 여섯개가져온다.
    return sorted(data.values(), key=lambda x: calcH(startpoint, x.index.values))[1:n+1]

def getParent(closedlist, index):
    path = []
    while index is not None:
        path.append(index)
        index = clos   edlist.get(index, None)
    return [data[i] for i in path[::-1]]


startIndex = 25479 # Hessle
endIndex = 8262 # Leeds

#namedTuple 생성 point_ID대신 원래는 ID였음 
Node = collections.namedtuple("Node", "point_ID F G H parentID".split())

h = calcH(startIndex, endIndex)
openlist = [(h, Node(startIndex, h, 0, h, None))] # heap
closedlist = {} # map visited nodes to parent

while len(openlist) >= 1:
    _, currentLocation = heapq.heappop(openlist)
    print(currentLocation)

    if currentLocation.ID in closedlist:
        continue
    closedlist[currentLocation.ID] = currentLocation.parentID

    if currentLocation.ID == endIndex:
        print("Complete")
        for p in getParent(closedlist, currentLocation.ID):
            print(p)
        break

    for other in getneighbors(currentLocation.ID):
        g = currentLocation.G + calcH(currentLocation.ID, other.ID)
        h = calcH(other.ID, endIndex)
        f = g + h
        heapq.heappush(openlist, (f, Node(other.ID, f, g, h, currentLocation.ID)))

Point_1789
Point_1379
Point_1316
Point_1373
Point_1372
Point_1317
Point_1382
Point_1380
Point_1381
Point_2072
Point_1298
Point_1296
Point_1378
Point_1292
Point_1293
Point_1295
Point_1805
Point_1374
Point_1321
Point_1834
Point_1503
Point_1320
Point_1371
Point_1318
Point_1323
Point_1322
Point_1319
Point_1437
Point_1502
Point_1438
Point_703
Point_2023
Point_1377
Point_704
Point_1504
Point_1445
Point_1375
Point_1376
Point_1439
Point_1637
Point_1385
Point_1326
Point_1297
Point_1294
Point_1892
Point_1722
Point_167
Point_278
Point_200
Point_279
Point_169
Point_173
Point_172
Point_2005
Point_247
Point_248
Point_253
Point_EP


In [206]:
# goodbye = pd.DataFrame(columns = ['위도', '경도'] )
# goodbye.append(points.loc[1:,'point_ID'].values)
points.loc[1:,'point_ID'].values
closed_list_id[57]

'Point_253'

In [207]:
# 역삼동 중심 = 역삼역 /  좌표 : 37.500742, 127.036891
to_map1 = folium.Map(location= sp, zoom_start=18)
a = closed_list
b 
# for n in range(0, len(a)) :|
#     if a['분류'][n] == '편의점' :
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='green')).add_to(to_map1)
#     elif a['분류'][n] == 'CCTV' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)        
#     elif a['분류'][n] == '보안등' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)  
#     else : 
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='black')).add_to(to_map1)        


for n in range(0, len(a)) :
      folium.CircleMarker([a[n][0], a[n][1]], popup= str(closed_list_seq[n]), radius = 3).add_to(to_map1)        

# 구분하기 쉽게 스타팅 지역은 빨간색 마커로 표시        
folium.Marker(sp, popup= 'SP', icon=folium.Icon(color='red')).add_to(to_map1)
folium.Marker(ep, popup= 'EP', icon=folium.Icon(color='black')).add_to(to_map1)
to_map1


In [208]:
df[df['point_ID'] == 'Point_248' ]
df[df['명칭'].str.contains('목화') ]

#closed_list_id

,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
1717,1795,Point_1795,127.043,GS25 역삼목화점,30,편의점,37.5089,75


In [211]:
dd = total_df[ total_df['point_ID'].isin(closed_list_id) ].sort_values('총밝기', ascending = False)
dd

,Unnamed: 0,point_ID,거리,밝기,총밝기,분류,명칭,위도,경도,경유여부
1789,1789,Point_1789,0,30,110,편의점,CU 역삼에스원점,37.506208,127.037951,0
1834,1834,Point_1834,0,25,110,CCTV,역삼동 655-13,37.506711,127.037181,0
1437,1437,Point_1437,0,5,110,보안등,역삼1동 661-3,37.505838,127.038051,0
1379,1379,Point_1379,0,5,105,보안등,역삼1동 656-12,37.506415,127.037943,0
1323,1323,Point_1323,0,5,105,보안등,역삼1동 655-13,37.506913,127.037214,0
1438,1438,Point_1438,0,5,105,보안등,역삼1동 660-33,37.505781,127.037757,0
1637,1637,Point_1637,0,30,100,편의점,GS25 역삼충현점,37.505715,127.037723,0
1316,1316,Point_1316,0,5,90,보안등,역삼1동 656-5,37.506599,127.037603,0
1503,1503,Point_1503,0,5,85,보안등,역삼1동 661-1,37.506062,127.037942,0
1317,1317,Point_1317,0,5,85,보안등,역삼1동 657-40,37.506480,127.037314,0


In [210]:
df[df['point_ID'] == 'Point_253' ]

,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
227,253,Point_253,127.043,역삼1동 684-27,5,보안등,37.5089,75
